In [ ]:
from __future__ import (absolute_import, division, print_function)
from functools import reduce
from operator import mul
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from pyneqsys.symbolic import SymbolicSys, linear_exprs
sp.init_printing()
prod = lambda x: reduce(mul, x)
names = 'Na+ Cl- NaCl'.split()
sp.__version__

Let's consider precipitation/dissolution of NaCl:
$$
NaCl(s) \leftrightharpoons Na^+(aq) + Cl^-(aq)
$$

In [ ]:
init_concs = iNa_p, iCl_m, iNaCl = [sp.Symbol('i_'+str(i), real=True, negative=False) for i in range(3)]
c = Na_p, Cl_m, NaCl = [sp.Symbol('c_'+str(i), real=True, negative=False) for i in range(3)]
stoichs = [[1, 1, -1]]
Na = [1, 0, 1]
Cl = [0, 1, 1]
e = [1, -1, 0]
preserv = [Na, Cl, e]
eq_constants = [Ksp] = [sp.Symbol('Ksp', real=True, positive=True)]

def get_f(x, params, solid):
    init_concs = params[:3] if solid else params[:2]
    eq_constants = params[3:]
    le = linear_exprs(preserv, x[:3] if solid else x[:2], linear_exprs(preserv, init_concs), rref=True)
    return le + ([Na_p*Cl_m - Ksp] if solid else [NaCl])

In [ ]:
f_true = get_f(c, init_concs + eq_constants, True)
f_false = get_f(c, init_concs + eq_constants, False)
f_true, f_false

In [ ]:
from pyneqsys.core import ConditionalNeqSys
cneqsys = ConditionalNeqSys([
            (lambda x, p: (x[0] + x[2]) * (x[1] + x[2]) > p[3],
             lambda x, p: x[2] >= 0)
        ],
        lambda conds: SymbolicSys(c, f_true if conds[0] else f_false,
                                  init_concs+eq_constants))

In [ ]:
c0, K = [0.5, 0.5, 0], [1]
params = c0 + K

In [ ]:
cneqsys.solve([0.5, 0.5, 0], params)

In [ ]:
cneqsys.solve([0.5, 0.5, 0], params)

In [ ]:
%matplotlib inline
ax_out = plt.subplot(1, 2, 1)
ax_err = plt.subplot(1, 2, 2)
xres, sols = cneqsys.solve_and_plot_series(
    c0, params, np.linspace(0, 3), 0, 'kinsol',
    {'ax': ax_out}, {'ax': ax_err}, fnormtol=1e-14)